In [1]:
import pandas as pd

In [4]:
#getting hourly avarage for noise

noise= pd.read_csv('noise_data/noise_data_complete.csv')


In [6]:
noise.info()
noise.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8593 entries, 0 to 8592
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   result_timestamp  8593 non-null   object 
 1   noise_level       8593 non-null   float64
 2   comp              8593 non-null   object 
dtypes: float64(1), object(2)
memory usage: 201.5+ KB


,result_timestamp,noise_level,comp
0,2022-01-01 00:00:00,55.88125,Intermediate
1,2022-01-01 01:00:00,58.05625,High
2,2022-01-01 02:00:00,51.57500,Intermediate
3,2022-01-01 03:00:00,48.67500,Intermediate
4,2022-01-01 04:00:00,46.12500,Intermediate


In [9]:
noise['result_timestamp'] = pd.to_datetime(noise['result_timestamp'])

# Extract the day of the week (0 = Monday, 6 = Sunday)
noise['DayOfWeek'] = noise['result_timestamp'].dt.dayofweek

In [10]:
noise.head()

,result_timestamp,noise_level,comp,DayOfWeek
0,2022-01-01 00:00:00,55.88125,Intermediate,5
1,2022-01-01 01:00:00,58.05625,High,5
2,2022-01-01 02:00:00,51.57500,Intermediate,5
3,2022-01-01 03:00:00,48.67500,Intermediate,5
4,2022-01-01 04:00:00,46.12500,Intermediate,5


In [14]:
# Group by the hour of the week (day of the week and hour of the day)
df_hourly_avg = noise.groupby(['DayOfWeek', noise['result_timestamp'].dt.hour])['noise_level'].mean().reset_index()
# Rename the columns for clarity
df_hourly_avg.columns = ['DayOfWeek', 'HourOfDay', 'AverageNoise']

# Preview the resulting dataset
print(df_hourly_avg)

     DayOfWeek  HourOfDay  AverageNoise
0            0          0     48.654779
1            0          1     45.111201
2            0          2     42.809967
3            0          3     41.655523
4            0          4     41.753627
..         ...        ...           ...
163          6         19     58.750359
164          6         20     58.612157
165          6         21     57.614714
166          6         22     55.675384
167          6         23     52.711250

[168 rows x 3 columns]


In [15]:
df_hourly_avg.to_csv('hourly_acg_noise.csv')

In [16]:
#importing packages
import streamlit as st
import pandas as pd
import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from datetime import timedelta
import requests
import numpy as np


  Using cached selenium-4.9.1-py3-none-any.whl (6.6 MB)
  Using cached trio_websocket-0.10.2-py3-none-any.whl (17 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached exceptiongroup-1.1.1-py3-none-any.whl (14 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
     -------------------------------------- 179.1/179.1 kB 1.1 MB/s eta 0:00:00
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
     ------------------------------------ 118.7/118.7 kB 695.2 kB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [18]:
#importing models

url_class = 'https://raw.githubusercontent.com/joaodpcm/MDA/main/classifier_trained_model.pkl'
response_class = requests.get(url_class)

url_reg= 'https://raw.githubusercontent.com/joaodpcm/MDA/main/Regressor_trained_model.pkl'
response_reg= requests.get(url_reg)

with open('classifier_trained_model.pkl', 'wb') as f:
    f.write(response_class.content)

with open('classifier_trained_model.pkl', 'rb') as f:
    rfc = pickle.load(f)

with open('Regressor_trained_model.pkl','wb') as f:
    f.write(response_reg.content)

with open('Regressor_trained_model.pkl', 'rb') as f:
    hgr = pickle.load(f)

UnpicklingError: invalid load key, '4'.

In [31]:
import datetime
print(datetime.version)

AttributeError: module 'datetime' has no attribute 'version'

In [22]:
with open('classifier_trained_model.pkl', 'rb') as f:
    rfc = pickle.load(f)

with open('Regressor_trained_model.pkl', 'rb') as f:
    hgr = pickle.load(f)

UnpicklingError: invalid load key, '4'.

In [ ]:
#importing avarage of the noise
df_hourly_avg=pd.read_csv('noise_data/hourly_acg_noise.csv')

# Retrieve the current date and time
current_time = datetime.now()

# Create a range of dates and times for the next 48 hours
time_range = pd.date_range(start=current_time, periods=48, freq='H')
# Filter the dataset for the next 48 hours
filtered_data = df_hourly_avg[
    (df_hourly_avg['DayOfWeek'] == current_time.weekday()) &
    (df_hourly_avg['HourOfDay'].isin(time_range.hour))
]

events=pd.read_csv('shaped_filter_tags_city2_EXAM.csv')
events['Time'] = pd.to_datetime(events['Time'])
events_48 = events[events['Time'].isin(time_range)]




In [ ]:
#loading unseen data
content = "https://weather.com/weather/hourbyhour/l/c097b546627cdff2da1e276cb9b2731055718a5e7270d777a92857a9701c7870"
response = requests.get(content)
soup = BeautifulSoup(response.content, 'html.parser')

temp_val = soup.findAll('div', attrs={'class':'DetailsTable--field--CPpc_'})


forecast = pd.DataFrame()
forecast['temp'] = [round((int(temp_val[i].text[-3:-1])-32 ) *5/9, 1) for i in list(np.array(range(288))) if i%6 == 0]
forecast['wind'] = [float(temp_val[i].text.split(' ')[1]) for i in list(np.array(range(288))) if i%6 == 1]
forecast['wind_direction'] = [temp_val[i].text.split(' ')[0] for i in list(np.array(range(288))) if i%6 == 1]
forecast['humidity'] = [int(temp_val[i].text[-3:-1]) for i in list(np.array(range(288))) if i%6 == 2]
forecast['cloud_cover'] = [int(temp_val[i].text.replace('Cloud Cover', '')[:-1]) for i in list(np.array(range(288))) if i%6 == 4]
forecast['rain'] = [int(temp_val[i].text.replace('Rain Amount', '').replace(' in', '')) for i in list(np.array(range(288))) if i%6 == 5]
weekday = [(datetime.now()+timedelta(hours=i)).weekday() for i in range(48)]
hour_of_day = [(datetime.now()+timedelta(hours=i)).hour for i in range(48)]
forecast['nameday'] = weekday
forecast['hour'] = hour_of_day
forecast['event_yes'] = False # This value has to be included by the user. So edit this. The value now is missing, but the model running, so even if nothing is provided, it will run
forecast['tag_category'] = 'No event' # This value has to be included by the user. So edit this
forecast['# of events'] = events_48['Events']



In [ ]:
#making prediction on unseen data
prediction_reg = hgr.predict(forecast)

prediction_class = rfc.predict(forecast)


In [ ]:

#making a graph for the classifier
colors = {"Low": "green", "Intermediate": "yellow", "High": "red"}
category_order = ["Low", "Intermediate", "High"]
fig_class = go.Figure(data=[go.Bar(x=forecast['hour'],
                            y=prediction_class,
                            marker=dict(color=[colors[level] for level in prediction_class]),
                            customdata=prediction_class,
                            hovertemplate="Hour: %{x}<br>Noise Level: %{customdata}<extra></extra>")])
fig_class.update_xaxes(title_text="Hours")
fig_class.update_yaxes(title_text="Noise Level", categoryorder="array", categoryarray=category_order)
fig_class.update_layout(title_text="Relative Noise Levels in the Next 48 Hours")


In [ ]:

#making a graph for the regressor
fig_reg = go.Figure()
fig_reg.add_trace(data=go.Scatter(x=forecast['hour'], y=prediction_reg, name='Noise Forecast'))
fig_reg.add_trace(go.Scatter(x=time_range, y=filtered_data['AverageNoise'], mode='lines', name='Average Noise',line=dict(dash='dash')))

fig_reg.update_xaxes(title_text="Hours")
fig_reg.update_yaxes(title_text="dB")
fig_reg.update_layout(title_text="Noise Forecast vs Avarage for the next 48 hours")


In [ ]:

#App
st.title("Noise forecast")

# Create a table with checkboxes for each hour
st.header("Are there any events on the next two days?")
selected_hours = st.multiselect('Select hours for the event', forecast['hour'], default=[])
# Update the 'Event' column based on the selected hours
forecast.loc[forecast['hour'].isin(selected_hours), 'Event'] = True
#Update tag
for hour in selected_hours:
    event_type = st.selectbox(f'Select event type for {hour}',['Party', 'Sports', 'Cultural', 'Pub Crawl'])
    forecast.loc[forecast['hour']==hour, 'tag_category']=event_type

st.header("Noise levels for the next 2 days")
st.plotly_chart(fig_class)
st.markdown("This graph shows a categorical prediction for the noise level for the next 48 hours relative to the usual noise levels on these hours. "
            '<span style="color:red">The red bars indicate hours that will be louder than usual.</span>'  
            '<span style="color:yellow">The yellow bars indicate hours that will be like the usual.</span>'
             '<span style="color:green">The green bars indicate hours that will be calmer than usual.</span>',unsafe_allow_html=True)
st.plotly_chart(fig_reg)
st.markdown('This graph shows the absolute levels of noise expected for the next 48 hours in the continuous line, and the avarage of these hours in the dotted line')